In [ ]:
import io
import csv
import numpy as np
import tensorflow as tf
import utils

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
training_data = [] # list of strings
filename = 'data/smos/smos_data_porter_balanced.txt'

In [ ]:
with open(filename, newline='') as datafile:
    data_reader = csv.reader(datafile, delimiter='\n')
    
    for row in data_reader:
        training_data.append(row[0])

In [ ]:
print(len(training_data))

In [ ]:
lengths = []
max = 0

for seq in training_data:
    lengths.append(len(seq.split()))
    if max < len(seq.split()):
        max = len(seq.split())

In [ ]:
print(max)

In [ ]:
labels = [] # list of strings
filename = 'data/smos/smos_labels_balanced.txt'

In [ ]:
with open(filename, newline='') as datafile:
    data_reader = csv.reader(datafile, delimiter='\n')
    
    for row in data_reader:
        labels.append(int(row[0]))

In [ ]:
# Load and split dataset
batch_size = 32
seed = 123

full_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/smos/train_balanced',
    batch_size=batch_size,
    label_mode='binary',
    seed=seed
)

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/smos/train_balanced',
    batch_size=batch_size,
    label_mode='binary',
    validation_split=0.2, 
    subset='training',
    seed=seed
)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/smos/train_balanced',
    batch_size=batch_size,
    label_mode='binary',
    validation_split=0.2, 
    subset='validation',
    seed=seed
)

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(2):
        print(label_batch[i].numpy(), text_batch.numpy()[i])

In [ ]:
# Configure the dataset for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
print('Vocab size: ', utils.vocabulary_size(training_data))
print('Avg seq length: ', sum(lengths) / len(lengths))

In [ ]:
# Vocabulary size and number of words in a sequence.
# Using ~avg sequence length of all sequences
sequence_length = 200
vocab_size = utils.vocabulary_size(training_data)

In [ ]:
# Use the text vectorization layer to normalize, split, and map strings to 
# integers. Note that the layer uses the custom standardization defined above. 
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = full_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [ ]:
embedding_dim = 8

# Embed vocabulary into embedding_dim dimensions.
# Embedding tutorial uses size, Text Classification tutorial uses size + 1
embedding_layer = tf.keras.layers.Embedding(vocab_size + 1, embedding_dim, name='embedding')

In [ ]:
model = Sequential([
    vectorize_layer,
    embedding_layer,
    #Dropout(0.2),
    GlobalAveragePooling1D(),
    #Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=15,
    callbacks=[tensorboard_callback]
)

In [ ]:
model.summary()

In [ ]:
original_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/smos/train',
    batch_size=batch_size,
    label_mode='binary',
    seed=seed
)

original_ds = original_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model.evaluate(original_ds)

In [ ]:
# Retrieve the trained word embeddings
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
# Save embeddings to disk
out_vec = io.open('data/smos_porter_vectors.tsv', 'w', encoding='utf-8')
out_meta = io.open('data/smos_porter_metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if  index == 0: continue # skip 0, it's padding.
    vec = weights[index] 
    out_vec.write('\t'.join([str(x) for x in vec]) + '\n')
    out_meta.write(word + '\n')
    
out_vec.close()
out_meta.close()